In [1]:
import numpy as np
import itertools as it

Number of qubits

In [2]:
m = 2

# Load transversal
transversal_inv = load('2_transversal_inv.sobj')

Functions

In [3]:
def base(M, n):
    # calculate the image of the base under a matrix M
    
    # make a set of all combinations of the first column and the last n columns (these correspond to X_1, Z_1,...,Z_n)
    s = []
    for i in range(n+1, 2*n):
        s.append(M[0:2*n, i])
    powerset = it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1))
    
    res = [vector(GF(2),2*n)]
        
    for i in powerset:
        v = vector(sum(i))     # calculate the sum of the elements of each combination (e.g IZZ = IZI + IIZ)
        res.append(v)
        
    return res

In [4]:
def pillars(M, n):
    # calculate the image of the pillars under a matrix M
    
    X1 = vector(M[0:2*n, 0])
    Z1 = vector(M[0:2*n, n])
    Y1 = X1 + Z1
    
    pI = base(M, n)
    pX = [(X1 + b) for b in pI]
    pY = [(Y1 + b) for b in pI]
    pZ = [(Z1 + b) for b in pI]
    
    return [pI, pX, pY, pZ]   

In [5]:
def tensor(A, n):
    # calculate the n fold tensor product of a matrix A
    
    kron = A
    count = 1
    while count < n:
        kron = np.kron(kron,A)
        count = count + 1
        
    if n == 2:
        res = np.reshape(kron, (4,4))
    elif n == 3:
        res = np.reshape(kron, (4,4,4)) 
    elif n == 4:
        res = np.reshape(kron, (4,4,4,4)) 
    elif n == 5:
        res = np.reshape(kron, (4,4,4,4,4)) 
        
    return res

In [6]:
def dist_stat(initial, M, n):
    # returns the success probability of an n-to-1 protocol M applied to an initial state
    pil = pillars(M, n)
    out = []
    for layer in pil:   
        coef = 0
        for elt in layer:
            if n == 2:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1])]
            if n == 3:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2])]
            if n == 4:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3])]
            if n == 5:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3]), \
                                    int(elt[4]) + 2*int(elt[n+4])]
        out.append(coef)   
    sp = sum(out)
    fid = out[0]/sp
    r = 1
    for i in out:
        r = r + float(i * log(i) / log(2))
    r = r*sp/n
    
    return sp, fid, r

In [11]:
def best_protocol(initial, transversal_inv, n, measure = "fidelity"):
    # calculates the best protocol from a dictionary of inverses of protocols (transversal_inv) applied to an initial state
    # as quality measures the fidelity ("fidelity"), success probability ("sucprob") or rate ("rate") can be chosen
              
    if measure == "sucprob":
        res = 0
        for key, M in transversal_inv.items():
            s = (dist_stat(initial, M, n))[0]
            if s > res:
                res = s
                opt_inv = M   
                
    if measure == "fidelity":
        res = 0
        for key, M in transversal_inv.items():
            f = (dist_stat(initial, M, n))[1]
            if f > res:
                res = f
                opt_inv = M
    
    if measure == "rate":
        res = -100
        for key, M in transversal_inv.items():
            r = (dist_stat(initial, M, n))[2]
            if r > res:
                res = r
                opt_inv = M       
    
    # Calculate inverse of optimal protocol        
    A = opt_inv[0:n, 0:n]
    B = opt_inv[0:n, n:2*n]
    C = opt_inv[n:2*n, 0:n]
    D = opt_inv[n:2*n, n:2*n]
    opt = block_matrix([[D.transpose(),-B.transpose()],[-C.transpose(),A.transpose()]], subdivide=False)
    
    return res, opt

In [ ]:
def sucprob_fid_lists(initial, transversal_inv, n):
    # calculate the possible distillation statistics (success probability & fidelity) of the protocols in a transversal
    # applied to an initial state
    
    fid = []
    sp = []
    fslist = []
    for key, M in transversal_inv.items():
        s = round(dist_stat(initial, M, n)[0],10)
        f = round(dist_stat(initial, M, n)[1],10)
        if (s,f) not in fslist:
            sp.append(s)
            fid.append(f)
            fslist.append((s,f))
    return sp, fid

Input states

In [ ]:
# isotropic input state

fid_in = [0.55, 0.575, 0.6, 0.625, 0.65, 0.675, 0.7, 0.725, 0.75, 0.775, 0.8, 0.825, 0.85, 0.875, 0.9, 0.925, 0.95, 0.975]
init_iso = []
for f in fid_in:
    init_iso.append(vector([f, (1-f)/3, (1-f)/3, (1-f)/3]))

In [8]:
# Bell diagonal state

p0 = np.random.uniform(0.5, 1)
p1 = np.random.uniform(0, 1-p0)
p2 = np.random.uniform(0, 1-p0-p1)
p3 = 1 - p0 - p1 - p2
lst = [p0, p1, p2, p3]
lst.sort()

init_bell = vector([lst[3], lst[2], lst[1], lst[0]])
print(init_bell)
#init_bell = vector([0.752604752074101, 0.10567340022267524, 0.0938043735551225, 0.04791747414810121])

Results

In [ ]:
for i in init_iso:
    print(sucprob_fid_lists(tensor(i, m), transversal_inv, m))

In [ ]:
print(sucprob_fid_lists(tensor(init_bell, m), transversal_inv, m))

In [ ]:
r_list = []
for i in init_iso:
    r_list.append((best_protocol(tensor(i, m), transversal_inv, m, measure = "rate"))[0])
print(r_list)

In [ ]:
# rate n = 1
r_list = []
for i in init_iso:
    r = 1
    for j in i:
        r = r + float(j * log(j) / log(2))
    r_list.append(r)
print(r_list)

In [ ]:
# rate DEJMPS
DEJ = matrix(GF(2),[[1, 0, 1, 0], [1, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1]])
r_list = []
for i in init_iso:
    r_list.append(dist_stat(tensor(i, 2), DEJ.inverse(), 2)[2])
print(r_list)
    

In [ ]:
print(best_protocol(tensor(init_bell, m), transversal_inv, m, measure = "fidelity"))